In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv
from holoviews import opts

hv.notebook_extension('bokeh', 'matplotlib')

In [ ]:
# Read information to connect to the database and put it in environment variables
import os
with open('ENVVARS.txt') as f:
    for line in f:
        parts = line.split('=')
        if len(parts) == 2:
            os.environ[parts[0]] = parts[1].strip()

In [ ]:
db_name = 'ticclat'
#db_name = 'ticclat_test'
os.environ['dbname'] = db_name

In [ ]:
from ticclat.dbutils import get_session, session_scope

Session = get_session(os.environ['user'], os.environ['password'], os.environ['dbname'])

In [ ]:
from ipywidgets import interact, interact_manual
from ticclat.queries import wordform_in_corpora_over_time

@interact_manual
def freq_data(word_form='regering'):
    if word_form != '':
        with session_scope(Session) as session:
            r = wordform_in_corpora_over_time(session, wf=word_form)
        r['normalized_term_frequency'] = r['term_frequency'] / r['num_words'] * 100.0
        data = hv.Dataset(hv.Table(r[['name', 'pub_year', 'normalized_term_frequency']]), 
                          ['name', 'pub_year'], ['normalized_term_frequency'])
        ndoverlay = data.select().to(hv.Curve, 'pub_year', 'normalized_term_frequency').overlay('name')
        ndoverlay.opts(opts.Curve(width=600))
        return ndoverlay
    else:
        print('Please type a word!')